In [1]:
import imageio
import matplotlib.pyplot as plt
import numpy

There configuration constants

In [2]:
a = 0
b = 10
teta = 20
dx = 0.1
dt = 0.01

u = 0
kappa = 1 / 2

s = u * dt / dx
r = kappa * dt / (dx ** 2)

There boundary conditions

In [3]:
def left_border(t):
    return 0

def right_border(t):
    return 0

There starting conditions

In [4]:
def generate_x_array(a, b, dx):
    return numpy.arange(a, b + dx, dx, dtype=numpy.float64)

def generate_starting_state(g, a, b, dx):
    arr = generate_x_array(a, b, dx)
    out = numpy.array([0] * len(arr))

    for i in range(len(arr)):
        out[i] = g(arr[i])

    return out

begin = generate_starting_state(lambda x: 1 if x < (a + b) // 2 else 0, a,  b, dx)

In [5]:
def explicit_downstream():
    T = [begin]

    t = dt
    while t <= teta:
        t_nplus1 = []
        for i in range(len(begin)):
            if i + 1 < len(begin):
                t_n_kplus1 = T[-1][i + 1]
            else:
                t_n_kplus1 = right_border(t)
      
            if i - 1 > 0:
                t_n_kminus1 = T[-1][i - 1]
            else:
                t_n_kminus1 = left_border(t)
      
            t_n_k = T[-1][i]

            t_nplus1.append(t_n_k - s * (t_n_kplus1 - t_n_k) + 
                            r * (t_n_kplus1 - 2 * t_n_k + t_n_kminus1))
    
        t += dt
    
        T.append(t_nplus1)
  
    return T

In [6]:
def explicit_upstream():
    T = [begin]

    t = dt
    while t <= teta:
        t_nplus1 = []
        for i in range(len(begin)):
            if i + 1 < len(begin):
                t_n_kplus1 = T[-1][i + 1]
            else:
                t_n_kplus1 = right_border(t)

            if i - 1 > 0:
                t_n_kminus1 = T[-1][i - 1]
            else:
                t_n_kminus1 = left_border(t)

            t_n_k = T[-1][i]

            t_nplus1.append(t_n_k - s * (t_n_k - t_n_kminus1) + 
                            r * (t_n_kplus1 - 2 * t_n_k + t_n_kminus1))

        t += dt

        T.append(t_nplus1)
  
    return T

In [7]:
def tridiagonal_matrix_solver(a, b, c, d):
    alpha = numpy.zeros(len(a), dtype=numpy.float64)
    betta = numpy.zeros(len(a), dtype=numpy.float64)
    gamma = numpy.zeros(len(a), dtype=numpy.float64)
    
    alpha[0] = -c[0] / b[0]
    betta[0] = d[0] / b[0]
    
    for i in range(1, len(a) - 1):
        tmp = b[i] + a[i] * alpha[i - 1]
        alpha[i] = -c[i] / tmp
        betta[i] = (d[i] - a[i] * betta[i - 1]) / tmp
    
    tmp = b[-1] + a[-1] * alpha[-2]
    betta[-1] = (d[-1] - a[-1] * betta[-2]) / tmp
    
    x = numpy.zeros(len(a), dtype=numpy.float64)
    
    x[-1] = betta[-1]
    for i in range(len(a) - 2, -1, -1):
        x[i] = alpha[i] * x[i + 1] + betta[i]
    
    return x

In [8]:
def implicit_downstream():
    T = [begin]

    t = dt
    while t <= teta:        
        a = [0] * len(begin)
        b = [0] * len(begin)
        c = [0] * len(begin)
        d = [0] * len(begin)
        
        for i in range(len(a)):
            a[i] = -r
            b[i] = 1 - s + 2 * r
            c[i] = s - r
            d[i] = T[-1][i]

        a[0] = 0
        c[0] = 0
        a[-1] = 0
        c[-1] = 0
        
        d[0] += left_border(t) * r
        d[-1] -= right_border(t) * (s - r)
        
        a = numpy.array(a)
        b = numpy.array(b)
        c = numpy.array(c)
        d = numpy.array(d)
        
        t += dt

        T.append(tridiagonal_matrix_solver(a, b, c, d))
  
    return T

In [9]:
def implicit_upstream():
    T = [begin]

    t = dt
    while t <= teta:        
        a = [0] * len(begin)
        b = [0] * len(begin)
        c = [0] * len(begin)
        d = [0] * len(begin)
        
        for i in range(len(a)):
            a[i] = - (s + r)
            b[i] = 1 + s + 2 * r
            c[i] = - r
            d[i] = T[-1][i]

        a[0] = 0
        c[0] = 0
        a[-1] = 0
        c[-1] = 0
        
        d[0] += left_border(t) * (r + s)
        d[-1] += right_border(t) * r
        
        a = numpy.array(a)
        b = numpy.array(b)
        c = numpy.array(c)
        d = numpy.array(d)
        
        t += dt

        T.append(tridiagonal_matrix_solver(a, b, c, d))
  
    return T

In [10]:
def nonsence():
    T = [begin]

    t = dt
    
    t_1 = []
    for i in range(len(begin)):
        if i + 1 < len(begin):
            t_n_kplus1 = T[-1][i + 1]
        else:
            t_n_kplus1 = right_border(t)

        if i - 1 > 0:
            t_n_kminus1 = T[-1][i - 1]
        else:
            t_n_kminus1 = left_border(t)

        t_n_k = T[-1][i]

        t_1.append(t_n_k - s * (t_n_k - t_n_kminus1) + 
                   r * (t_n_kplus1 - 2 * t_n_k + t_n_kminus1))
    T.append(t_1)
    t += dt
    
    while t <= teta:
        t_nplus1 = []
        for i in range(len(begin)):
            if i + 1 < len(begin):
                t_n_kplus1 = T[-1][i + 1]
            else:
                t_n_kplus1 = right_border(t)

            if i - 1 > 0:
                t_n_kminus1 = T[-1][i - 1]
            else:
                t_n_kminus1 = left_border(t)

            t_nminus2_k = T[-2][i]

            t_nplus1.append(t_nminus2_k - s * (t_n_kplus1 - t_n_kminus1))

        t += dt

        T.append(t_nplus1)
  
    return T

In [11]:
def animated_plots(T, method_name, pictures_count):
    x = generate_x_array(a, b, dx)

    min_T = numpy.min(T)
    max_T = numpy.max(T)
    
    indexes = numpy.arange(0, len(T), len(T) // pictures_count)
    
    pictures = []
    for i in indexes:
        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(x, T[i])

        ax.set_ylim(min_T - 0.1, max_T + 0.1)

        fig.canvas.draw()
        picture = numpy.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        picture = picture.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close(fig)

        pictures.append(picture)

    imageio.mimsave('./animated_plot_' + method_name + '.gif', pictures, fps=24)

In [12]:
animated_plots(explicit_downstream(), "explicit_downstream", 400)

In [13]:
animated_plots(explicit_upstream(), "explicit_upstream", 400)

In [14]:
animated_plots(implicit_downstream(), "implicit_downstream", 400)

In [15]:
animated_plots(implicit_upstream(), "implicit_upstream", 400)

In [ ]:
u, kappa = kappa, u
s = u * dt / dx
r = kappa * dt / (dx * dx)
print('s = %f, r = %f' % (s, r))
animated_plots(nonsence(), "nonsence", 400)

s = 0.050000, r = 0.000000
